Define cv2_imshow from google collab because it wont install

In [3]:
import cv2
from IPython import display
import PIL

def cv2_imshow(a):
  """A replacement for cv2.imshow() for use in Jupyter notebooks.

  Args:
    a : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
      (N, M, 3) is an NxM BGR color image. shape (N, M, 4) is an NxM BGRA color
      image.
  """
  a = a.clip(0, 255).astype('uint8')
  # cv2 stores colors as BGR; convert to RGB
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display.display(PIL.Image.fromarray(a))


# Configure our Model

using PyTorch download a pre-trained YOLOv5 model for image processing

In [4]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.5 #NMS Confidence Threshold
model.multi_label = False
model.classes = [0] # Detect people only
model.max_det = 1000

/home/athielking/src/person-recognizer/.venv/lib/python3.8/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/athielking/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-9-12 Python-3.8.10 torch-2.0.1+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 45.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape..

# Process Images

loop through our directory and see what images have people in them

In [9]:
import os

directory_path = '/mnt/c/Users/athie/Pictures'
for filename in os.listdir(directory_path):
    if filename.endswith((".jpg", ".jpeg", ".png", ".gif")):

        image_path = os.path.join(directory_path, filename)
        
        # Open and process the image using OpenCV
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB format
       
        # Perform image processing or analysis here
        with torch.no_grad():
            results = model(img)
            results.render()
            dataFrame = results.pandas().xyxy[0]
            print(dataFrame)
            # cv2_imshow(results.ims[0])
            # p = results.pandas().xyxy[0]
            # for idx, row in p.iterrows():
            #     print("PERSON FOUND")
            #     print(image_path)

         xmin        ymin         xmax         ymax  confidence  class    name
0  575.420166  450.216278  1242.025757  1598.919678    0.840980      0  person
1   50.831242  105.015335   682.140137  1600.000000    0.696541      0  person
          xmin         ymin         xmax         ymax  confidence  class  \
0   120.656799  2609.779785  1210.084961  6201.459961    0.663172      0   
1  2370.112305  2907.695068  2869.566162  3828.691162    0.606550      0   
2   447.991119   787.123779  2354.145508  6496.900391    0.523442      0   

     name  
0  person  
1  person  
2  person  
          xmin         ymin         xmax         ymax  confidence  class  \
0  1178.602295   866.873657  3752.710938  5972.000000    0.898618      0   
1  3722.139404  3340.661865  4478.321289  5940.788574    0.818310      0   

     name  
0  person  
1  person  
          xmin         ymin         xmax         ymax  confidence  class  \
0     4.422177  1773.255127   590.005005  4462.509277    0.916027    